In [1]:
from leaguedata.data import get_history_dict

history = get_history_dict()

In [2]:
from jax.tree_util import tree_map
from statsmodels.tsa.stattools import acf

autocorrelation = tree_map(lambda x : acf(x, nlags = 10)[1:], history)

In [3]:
import numpy as np 

simulation_size = 99
p = 1/2
p_streak = 0.8

history_random_list = []
history_correlated_list = []
history_anticorrelated_list = []

for i in range(1000):

    history_random = [0]
    history_correlated = [0]
    history_anticorrelated = [0]

    for _ in range(simulation_size):
        
        #previous = history_streak[-1]
        history_random.append(np.random.binomial(1, p))
        history_correlated.append(np.random.binomial(1, p_streak if history_correlated[-1] else 1 - p_streak))
        history_anticorrelated.append(np.random.binomial(1, p_streak if not history_anticorrelated[-1] else 1 - p_streak))

    history_random_list.append(history_random)
    history_correlated_list.append(history_correlated)
    history_anticorrelated_list.append(history_anticorrelated)

history_random = np.apply_along_axis(acf, 1, np.asarray(history_random_list), nlags = 10)[..., 1:]
history_correlated = np.apply_along_axis(acf, 1, np.asarray(history_correlated_list), nlags = 10)[..., 1:]
history_anticorrelated = np.apply_along_axis(acf, 1, np.asarray(history_anticorrelated_list), nlags = 10)[..., 1:]

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assume the data setup as previously mentioned
# Separate matrices
random_matrix = np.asarray(history_random_list)[0:10, 1:60]
correlated_matrix = np.asarray(history_correlated_list)[0:10, 1:60]
anticorrelated_matrix = np.asarray(history_anticorrelated_list)[0:10, 1:60]

# Define colors for each series for consistency
color_random = 'rgba(0,100,80,0.5)'
color_correlated = 'rgba(0,176,246,0.5)'
color_anticorrelated = 'rgba(231,107,243,0.5)'

x_values = np.arange(10) + 1

# Adjusted plot code with shared color and linked visibility
# Create traces for the medians with legendgroup
trace_random = go.Scatter(
    x=x_values,
    y=np.median(history_random, axis=0),
    mode='lines',
    name='Random',
    line=dict(color=color_random),
    legendgroup='random'
)

trace_correlated = go.Scatter(
    x=x_values,
    y=np.median(history_correlated, axis=0),
    mode='lines',
    name='Correlated',
    line=dict(color=color_correlated),
    legendgroup='correlated'
)

trace_anticorrelated = go.Scatter(
    x=x_values,
    y=np.median(history_anticorrelated, axis=0),
    mode='lines',
    name='Anti-correlated',
    line=dict(color=color_anticorrelated),
    legendgroup='anticorrelated'
)

# Create traces for the percentiles fill with the same legendgroup and matching color
fill_random = go.Scatter(
    x=np.concatenate((x_values, x_values[::-1])),
    y=np.concatenate((
        np.percentile(history_random, 5, axis=0), 
        np.percentile(history_random, 95, axis=0)[::-1]
    )),
    fill='toself',
    fillcolor=color_random,
    line=dict(color='rgba(255,255,255,0)'),
    name='Random Fill',
    showlegend=False,
    legendgroup='random'
)

fill_correlated = go.Scatter(
    x=np.concatenate((x_values, x_values[::-1])),
    y=np.concatenate((
        np.percentile(history_correlated, 5, axis=0),
        np.percentile(history_correlated, 95, axis=0)[::-1]
    )),
    fill='toself',
    fillcolor=color_correlated,
    line=dict(color='rgba(255,255,255,0)'),
    name='Correlated Fill',
    showlegend=False,
    legendgroup='correlated'
)

fill_anticorrelated = go.Scatter(
    x=np.concatenate((x_values, x_values[::-1])),
    y=np.concatenate((
        np.percentile(history_anticorrelated, 5, axis=0),
        np.percentile(history_anticorrelated, 95, axis=0)[::-1]
    )),
    fill='toself',
    fillcolor=color_anticorrelated,
    line=dict(color='rgba(255,255,255,0)'),
    name='Anti-correlated Fill',
    showlegend=False,
    legendgroup='anticorrelated'
)

# Create subplots with 3 rows for matrices and 1 big column for correlation plots
fig = make_subplots(
    rows=3, cols=2,
    column_widths=[0.65, 0.35],
    specs=[[{"type": "scatter", "rowspan":3}, {"type": "heatmap", "colspan": 1}],
           [None, {"type": "heatmap", "colspan": 1}],
           [None, {"type": "heatmap", "colspan": 1}]],
    subplot_titles=("", "Random", "Correlated", "Anti-correlated")
)

# Add heatmaps to the first column
fig.add_trace(
    go.Heatmap(
        z=random_matrix,
        colorscale=[[0, 'mediumvioletred'], [1, 'mediumseagreen']],
        showscale=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Heatmap(
        z=correlated_matrix,
        colorscale=[[0, 'mediumvioletred'], [1, 'mediumseagreen']],
        showscale=False
    ),
    row=2, col=2
)

fig.add_trace(
    go.Heatmap(
        z=anticorrelated_matrix,
        colorscale=[[0, 'mediumvioletred'], [1, 'mediumseagreen']],
        showscale=False
    ),
    row=3, col=2
)

# Add correlation plots to the second column spanning all three rows
# Add the fill traces first so they're behind the median traces to the second column
fig.add_trace(fill_random, row=1, col=1)
fig.add_trace(fill_correlated, row=1, col=1)
fig.add_trace(fill_anticorrelated, row=1, col=1)

# Add the median traces to the second column
fig.add_trace(trace_random, row=1, col=1)
fig.add_trace(trace_correlated, row=1, col=1)
fig.add_trace(trace_anticorrelated, row=1, col=1)
# Update layout for a cleaner look
fig.update_layout(
    #title_text="Vertical Matrix with Right-Side Correlation Traces",
    showlegend=True
)

# Hide tick labels on heatmaps
fig.update_xaxes(showticklabels=False, row=1, col=2)
fig.update_xaxes(showticklabels=False, row=2, col=2)
fig.update_xaxes(showticklabels=False, row=3, col=2)
fig.update_yaxes(showticklabels=False, row=1, col=2)
fig.update_yaxes(showticklabels=False, row=2, col=2)
fig.update_yaxes(showticklabels=False, row=3, col=2)
fig.update_yaxes(range=(-1, 1), row=1, col=1)
fig.update_xaxes(range=(1, 8), row=1, col=1)

fig['layout']['xaxis1']['title']="Lag between games"
fig['layout']['yaxis1']['title']="Correlation coefficient"

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    width=600, height=800/3,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.6
    )
)

fig.update_xaxes(showgrid=True, minor=dict(showgrid=True), row=1, col=1)
fig.update_yaxes(showgrid=True, minor=dict(showgrid=True), row=1, col=1, title_standoff = 0)

# Show the plot
fig.show()

with open("docs/loserQ/assets/simulated_correlation.json", "w") as f:
    f.write(fig.to_json())

In [6]:
import jax
import jax.numpy as jnp
from jax.random import PRNGKey
from leaguedata.model import DTMCModel

markov = DTMCModel(1)

def single_history(key, probs_keys, number_of_games):
    probs = jnp.empty((2 ** 1))
    
    for i, state in enumerate(markov.get_states()):
        probs = probs.at[i].set(probs_keys[sum(state)])

    return markov.build_process(number_of_games, probs=probs).sample(1, seed=key)[0]

probs_keys = {
1.: np.random.normal(50.54/100, 0.14/100, size=3100),
0.: np.random.normal(49.81/100, 0.14/100, size=3100)
}

keys = jax.random.split(PRNGKey(72), 3100)
history_categorical = np.asarray(
    jax.vmap(lambda key, probs: single_history(key, probs, 100)
             )(keys, probs_keys))

history = np.apply_along_axis(markov.categorical_serie_to_binary, 1, history_categorical)
simulated_autocorrelation = np.apply_along_axis(lambda x : acf(x, nlags = 10)[1:], 1, history)

history = np.apply_along_axis(markov.categorical_serie_to_binary, 1, history_categorical)
simulated_autocorrelation = np.apply_along_axis(lambda x : acf(x, nlags = 10)[1:], 1, history)

In [18]:
from leaguedata.model import generate_obvious_loser_q

mock_loserq_autocorrelation = np.apply_along_axis(lambda x : acf(x, nlags = 10)[1:], 1, generate_obvious_loser_q(number_of_players=3100))

#'rgba(255, 191, 0, 0.6)'

In [32]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import random 

fig = go.Figure()

once = True
    
fill_autocorrelation = go.Scatter(
    x=np.concatenate((x_values, x_values[::-1])),
    y=np.concatenate((
        np.percentile(simulated_autocorrelation, 5, axis=0),
        np.percentile(simulated_autocorrelation, 95, axis=0)[::-1]
    )),
    fill='toself',
    fillcolor='rgba(60,179,113,0.6)',
    line=dict(color='rgba(60,179,113,0.9)'),
    name='Best-fit',
    showlegend=True,
    legendgroup='simulated_autocorr'
)

median_autocorrelation = go.Scatter(
    x=x_values,
    y=np.median(simulated_autocorrelation, axis=0),
    mode='lines',
    name='Random',
    line=dict(color='rgba(60,179,113,0.9)'),
    showlegend=False,
    legendgroup='simulated_autocorr'
)

fig.add_trace(fill_autocorrelation)
fig.add_trace(median_autocorrelation)

fill_loserq = go.Scatter(
    x=np.concatenate((x_values, x_values[::-1])),
    y=np.concatenate((
        np.percentile(mock_loserq_autocorrelation, 5, axis=0),
        np.percentile(mock_loserq_autocorrelation, 95, axis=0)[::-1]
    )),
    fill='toself',
    fillcolor='rgba(255, 191, 0, 0.6)',
    line=dict(color='rgba(255, 191, 0, 0.9)'),
    name='Mock LoserQ',
    showlegend=True,
    legendgroup='simulated_loserq'
)

median_loserq = go.Scatter(
    x=x_values,
    y=np.median(mock_loserq_autocorrelation, axis=0),
    mode='lines',
    name='Random',
    line=dict(color='rgba(255, 191, 0, 0.6)'),
    showlegend=False,
    legendgroup='simulated_loserq'
)

fig.add_trace(fill_loserq)
fig.add_trace(median_loserq)

for i in random.choices(np.arange(3100), k=80):
    
    autocorr_trace = go.Scatter(
        x=x_values,
        y=autocorrelation[i],
        mode='lines',
        name=f'Random player',
        legendgroup='autocorr_sample',
        showlegend=once,
        line=dict(color='rgba(108, 122, 137, 0.1)'),
        hoverinfo='skip'
    )
    
    once = False
    
    fig.add_trace(autocorr_trace)

fig['layout']['xaxis1']['title']="Lag between games"
fig['layout']['yaxis1']['title']="Correlation coefficient"

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    width=600, height=800/3,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.9
    )
)

fig.update_xaxes(showgrid=True, minor=dict(showgrid=True))
fig.update_yaxes(range=[-1,1])
fig.show()

with open("docs/loserQ/assets/true_data_correlation.json", "w") as f:
    f.write(fig.to_json())